In [10]:
from langchain_ollama import OllamaLLM
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage,AIMessage,SystemMessage,trim_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START,MessagesState,StateGraph

In [11]:
model=OllamaLLM(model="llama3.2")
prompt_template=ChatPromptTemplate.from_messages([
    SystemMessage(content="You are Alex, a personal AI assistant. Your purpose is to assist, provide accurate information, and engage in meaningful conversations. Always adapt your tone to be friendly, empathetic, and respectful. Respond concisely and directly unless further clarification or detail is requested. Prioritize understanding the user's preferences, maintaining context, and ensuring a helpful, personalized experience."),
    MessagesPlaceholder(variable_name="messages"),
])

In [12]:
# Define graph
workflow=StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state:MessagesState):
    prompt=prompt_template.invoke(state)
    result=[]
    for chunk in model.stream(prompt):
        result.append(chunk)
        print(chunk,end="")
    result="".join(result)
    # response=[AIMessage(model.invoke(prompt))]
    response=[AIMessage(result)]
    state["messages"].append(AIMessage(content=response[0].content))
    return {"message":state["messages"]}

# Define the (single) node in the graph
workflow.add_edge(START,"model")
workflow.add_node("model",call_model)

# Add Memory
memory=MemorySaver()
app = workflow.compile(checkpointer=memory)

In [13]:
config={"configurable":{"thread_id":"abc123"}}

In [14]:
def chat(query):
    input_messages = [HumanMessage(query)]
    output = app.invoke({"messages": input_messages}, config)
    return output

In [9]:
out=chat("what is your name?")

I think we're having a fun loop! My name is Alex, and I'll stick with my original introduction since our conversation just started. Would you like me to recall any specific information about yourself or help you with something else, Aayush?

In [99]:
out

{'messages': [HumanMessage(content='what is your name?', additional_kwargs={}, response_metadata={}, id='52fd1158-9860-4f1c-8b7b-1dbd91e516b6'),
  AIMessage(content="Nice to meet you! My name is Alex, and I'm here to help you with anything you need. How can I assist you today?", additional_kwargs={}, response_metadata={})]}

In [7]:
out=chat("my name is aayush dangol")

Nice to meet you, Aayush! It's great that we're starting our conversation. Is there something specific on your mind, or would you like some suggestions on how I can help you?

In [8]:
out=chat("whats my name?")

Nice to confirm names, Aayush! To clarify, your correct spelling is "Aayush Dangol", and you mentioned it earlier that the spelling "aayush" was used. If you'd prefer to use either version, I'm here for you!